In [8]:
!pip install scikit-learn lightgbm scikeras tensorflow pandas numpy matplotlib seaborn

In [74]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
#Sklearn imports
from sklearn.metrics import make_scorer, recall_score, precision_score, roc_auc_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier

from lightgbm import LGBMClassifier

#Add the parent directory to access ENV variables
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#Import of necessary paths ( GDC data Path and Dataset folder)
from config import THYROID_PATH, MODEL_PATH, RESULTS_PATH

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [16]:
#definition of scoring metrics
scoring={
          'acc': 'accuracy',
          'roc': make_scorer(roc_auc_score),
          'recall0': make_scorer(recall_score, average = None,labels=[0]),
          'recall1': make_scorer(recall_score, average = None,labels=[1]),
          'precision0': make_scorer(precision_score, average = None,labels=[0],zero_division=0),
          'precision1': make_scorer(precision_score, average = None,labels=[1],zero_division=0),
          'f0': make_scorer(f1_score,average=None,labels = [0]),
          'f1': make_scorer(f1_score,average=None,labels = [1]),
           }

imputer = SimpleImputer(strategy='constant')

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1,random_state=2024)

def save_report(report,folder,name,experiment="5fold_Repeated"):
    df = pd.DataFrame(report).transpose()
    savepath = os.path.join(folder,f'{name}_{experiment}.csv')
    df.to_csv(savepath)

In [17]:
models = {}
models['Dummy_prior'] = DummyClassifier(strategy="prior")
models['Dummy_prob']= DummyClassifier(strategy="stratified")
models['Logistic_elastic'] = LogisticRegression(penalty='elasticnet',solver='saga',class_weight='balanced', C=0.02, max_iter=200,l1_ratio=0.7)
models['QDA'] = QuadraticDiscriminantAnalysis()
models['SVC'] = SVC(C=0.2,class_weight='balanced') 
models['RF'] = RandomForestClassifier(50, max_depth=3,max_features='log2')

def get_uncompiled_model(reset_last_layer=False):
    
    model = keras.models.load_model(os.path.join(MODEL_PATH,'pan-cancer-solid-only'))
    if(reset_last_layer):
        output_follicolar= keras.layers.Dense(1, activation='sigmoid',name='output_follicolar')(model.layers[-2].output)
        model = keras.models.Model(inputs=model.input, outputs = [output_follicolar])
    return model

def get_compiled_model(metrics=None,reset_last_layer=False):
    
    if(metrics is None):
        metrics = [
              keras.metrics.TruePositives(name='tp'),
              keras.metrics.FalsePositives(name='fp'),
              keras.metrics.TrueNegatives(name='tn'),
              keras.metrics.FalseNegatives(name='fn'), 
              keras.metrics.BinaryAccuracy(name='accuracy'),
              keras.metrics.Precision(name='precision'),
              keras.metrics.Recall(name='recall'),
              keras.metrics.AUC(name='auc'),
              keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
        ]
        
    model = get_uncompiled_model(reset_last_layer)
    model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(3e-5),metrics=metrics)
    
    return model

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

kwargs = dict(
    model=get_compiled_model,
    epochs=40,
    verbose=True,
    batch_size = 8,
    callbacks = [early_stopping_cb],
    shuffle=True,
    #validation_split=0.2,
    fit__class_weight = None
)

# Unfiltered Cancer (Cancer vs Normal)

In [18]:
UnfilteredCancerPath = Path(THYROID_PATH,'UnfilteredCancerData.npy')
npzfiles = np.load(UnfilteredCancerPath,allow_pickle=True)

X = npzfiles['X']
y = npzfiles['y']

output_folder = Path(RESULTS_PATH,'UnfilteredCancer/')
output_folder.mkdir(exist_ok=True) #Create output folder if it does not exist 

In [19]:
pos = np.sum(y)
total = len(y)
neg  = total-pos

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

kwargs['fit__class_weight'] = class_weight

models['NeuralNetwork']=KerasClassifier(**kwargs)

In [20]:
from sklearn.feature_selection import VarianceThreshold

# 1. Define paths
UnfilteredCancerPath = Path(THYROID_PATH, 'UnfilteredCancerData.npy')

# 2. Load the data
print(f"Loading data from {UnfilteredCancerPath}...")
npzfiles = np.load(UnfilteredCancerPath, allow_pickle=True)

X = npzfiles['X']
y = npzfiles['y']
print(f"Original shape: {X.shape}")

# 3. Optimization: Remove features with zero variance (all NaNs or constant values)
# This removes the 485,000+ columns that were causing the Imputer to crash/hang.
selector = VarianceThreshold() 
X = selector.fit_transform(X)

# 4. Finalize shapes and folders
print(f"New shape after removing empty features: {X.shape}")

output_folder = Path(RESULTS_PATH, 'UnfilteredCancer/')
output_folder.mkdir(exist_ok=True, parents=True)

Loading data from /home/Capstone_Team78/Dataset/Datasets/UnfilteredCancerData.npy...
Original shape: (517, 485577)


/home/Capstone_Team78/venv/lib/python3.12/site-packages/sklearn/feature_selection/_variance_threshold.py:114: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
/home/Capstone_Team78/venv/lib/python3.12/site-packages/sklearn/feature_selection/_variance_threshold.py:122: RuntimeWarning: All-NaN slice encountered
  self.variances_ = np.nanmin(compare_arr, axis=0)


New shape after removing empty features: (517, 421318)


# Unfiltered Subtype (FvPTC vs CvPTC)

In [87]:
import keras
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

def get_keras3_model():
    # The exact path to your model folder
    MODEL_PATH = '/home/Capstone_Team78/Dataset/Models/pan-cancer-solid-only'
    
    # Keras 3 requires an explicit Input layer
    # We use the target dimension of your X_nn features
    inputs = keras.Input(shape=(485577,))
    
    # Genomic models usually expect (batch, features, 1)
    x = keras.layers.Reshape((485577, 1))(inputs)
    
    # Use the TFSMLayer bridge
    tfs_layer = keras.layers.TFSMLayer(MODEL_PATH, call_endpoint='serving_default')
    x = tfs_layer(x)
    
    # TFSMLayer outputs a dictionary. We grab the first output tensor.
    if isinstance(x, dict):
        x = x[list(x.keys())[0]]
        
    # Add the final classification head
    outputs = keras.layers.Dense(2, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Update your models dictionary
models['NeuralNetwork'] = KerasClassifier(
    model=get_keras3_model,
    epochs=5,
    batch_size=4,
    verbose=1
)

In [22]:
pos = np.sum(y)
total = len(y)
neg  = total-pos

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

kwargs['fit__class_weight'] = class_weight

models['NeuralNetwork']=KerasClassifier(**kwargs, reset_last_layer=True)

In [63]:
import numpy as np
from sklearn.model_selection import cross_validate

X_ready = np.nan_to_num(X, nan=0.0)

print(f"--- Starting Fine-Tuning: NeuralNetwork (Binary Class Fix) ---")

name = 'NeuralNetwork'
clf = models[name]

try:
    report = cross_validate(
        clf, 
        X_ready, 
        y, 
        cv=cv, 
        scoring=scoring, 
        n_jobs=1,
        error_score='raise'
    )
    save_report(report, output_folder, f"{name}_Finetuned")
    print(f"\nSuccessfully finished training all folds!")
except Exception as e:
    print(f"\nFine-tuning failed. Error Details:\n{e}")

--- Starting Fine-Tuning: NeuralNetwork (Binary Class Fix) ---
Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.7799 - loss: 0.5655
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.7799 - loss: 0.5653
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.7745 - loss: 0.5651
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.7717 - loss: 0.5647
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.7690 - loss: 0.5644
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 271ms/step
Epoch 1/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.2276 - loss: 1.0644
Epoch 2/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.2304 - loss: 1.0616
Epoch 3/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.2304 - loss: 1.0588
Epoch 4/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.2249 - loss: 1.0559
Epoch 5/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.2249 - loss: 1.0527
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 283ms/step
Epoch 1/5
93/93

# Filtered Cancer (Normal vs Cancer)


In [64]:
FilteredCancerPath = Path(THYROID_PATH,'FilteredCancerData.npy')
npzfiles = np.load(FilteredCancerPath,allow_pickle=True)

X = npzfiles['X']
y = npzfiles['y']
X_nn = npzfiles['X_nn'] #Zero-padded dataset for Neural Net Dimensionality

output_folder = Path(RESULTS_PATH,'FilteredCancer/')
output_folder.mkdir(exist_ok=True) #Create output folder if it does not exist 

In [65]:
pos = np.sum(y)
total = len(y)
neg  = total-pos

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

kwargs['fit__class_weight'] = class_weight

models['NeuralNetwork']=KerasClassifier(**kwargs)

In [73]:
import warnings
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

# Silence the syntax and convergence warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

print(f"Starting Execution on {X.shape[0]} samples...")

for name, clf in models.items():
    print(f"\n--- Processing: {name} ---")
    
    try:
        if name == 'NeuralNetwork':
            # FIX: Neural Network needs exactly 485577 features.
            # We fill NaNs with 0.0 manually to prevent the imputer from dropping columns.
            X_nn_fixed = np.nan_to_num(X_nn, nan=0.0)
            
            # We skip the pipeline (selector/imputer) to ensure the shape stays constant.
            report = cross_validate(
                clf, 
                X_nn_fixed, 
                y, 
                cv=cv, 
                scoring=scoring, 
                n_jobs=1
            )
        else:
            # Traditional models use the Pipeline logic
            if name == 'QDA':
                selector = SelectFromModel(
                    LogisticRegression(penalty='l1', solver='liblinear', C=0.1),
                    max_features=40
                )
            else:
                selector = 'passthrough'
                
            pipe = Pipeline(steps=[
                ('imputation', imputer),
                ('selector', selector),
                ('classifier', clf)
            ])
            
            report = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=1)
            
        save_report(report, output_folder, name)
        print(f"SUCCESS: {name} completed.")

    except Exception as e:
        print(f"FAILED {name}: {e}")

print("\n--- All models processed ---")

Starting Execution on 517 samples...

--- Processing: Dummy_prior ---
SUCCESS: Dummy_prior completed.

--- Processing: Dummy_prob ---
SUCCESS: Dummy_prob completed.

--- Processing: Logistic_elastic ---
SUCCESS: Logistic_elastic completed.

--- Processing: QDA ---
SUCCESS: QDA completed.

--- Processing: SVC ---
SUCCESS: SVC completed.

--- Processing: RF ---
SUCCESS: RF completed.

--- Processing: NeuralNetwork ---
Epoch 1/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 206s 2s/step - accuracy: 0.1090 - loss: 0.8910
Epoch 2/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.1090 - loss: 0.8890
Epoch 3/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 209s 2s/step - accuracy: 0.1090 - loss: 0.8870
Epoch 4/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.1090 - loss: 0.8850
Epoch 5/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.1090 - loss: 0.8831
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 279ms/step
Epoch 1/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 197s 2s/step - accuracy: 0.1065 - loss: 1.6174
Epoch 2/5
104/104 ━━━━━

# Filtered Subtype (FvPTC vs CvPTC)

In [75]:
FilteredSubtypePath = Path(THYROID_PATH,'FilteredSubtypeData.npy')
npzfiles = np.load(FilteredSubtypePath,allow_pickle=True)

X = npzfiles['X']
y = npzfiles['y']
X_nn = npzfiles['X_nn'] #Zero-padded dataset for Neural Net Dimensionality

output_folder = Path(RESULTS_PATH,'FilteredSubtype/')
output_folder.mkdir(exist_ok=True) #Create output folder if it does not exist 

In [76]:
pos = np.sum(y)
total = len(y)
neg  = total-pos

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

kwargs['fit__class_weight'] = class_weight

models['NeuralNetwork']=KerasClassifier(**kwargs, reset_last_layer=True)

In [80]:
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np

for name, clf in models.items():
    print(f"--- Processing: {name} ---")
    
    # 1. Selector Logic
    if name == 'QDA':
        # SelectKBest guarantees we don't get 0 features
        selector = SelectKBest(score_func=f_classif, k=50) 
    else:
        selector = 'passthrough'
    
    pipe = Pipeline(steps=[
        ('imputation', imputer),
        ('selector', selector),
        ('classifier', clf)
    ])  
    
    try:
        if name == 'NeuralNetwork':
            # Pre-fill NaNs so the model doesn't see them
            X_nn_clean = np.nan_to_num(X_nn, nan=0.0)
            # Pass directly to avoid pipeline issues
            report = cross_validate(clf, X_nn_clean, y, cv=cv, scoring=scoring, n_jobs=1)
        else:
            report = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=1)
            
        save_report(report, output_folder, name)
        print(f"SUCCESS: {name} finished.")
        
    except Exception as e:
        print(f"FAILED: {name} error: {e}")

print("\n--- All Tasks Finished ---")

--- Processing: Dummy_prior ---
SUCCESS: Dummy_prior finished.
--- Processing: Dummy_prob ---
SUCCESS: Dummy_prob finished.
--- Processing: Logistic_elastic ---
SUCCESS: Logistic_elastic finished.
--- Processing: QDA ---
SUCCESS: QDA finished.
--- Processing: SVC ---
SUCCESS: SVC finished.
--- Processing: RF ---
SUCCESS: RF finished.
--- Processing: NeuralNetwork ---
Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.7799 - loss: 0.5285
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.7799 - loss: 0.5279
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.7799 - loss: 0.5274
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.7799 - loss: 0.5278
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 177s 2s/step - accuracy: 0.7799 - loss: 0.5275
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 271ms/step
Epoch 1/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.7778 - loss: 0.5387
Epoch 2/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.7778 - loss: 0.5340
E

# Differential Methylation Cancer (Cancer vs Normal)

In [81]:
DifferentialCancerPath = Path(THYROID_PATH,'DifferentialCancerData.npy')
npzfiles = np.load(DifferentialCancerPath,allow_pickle=True)

X = npzfiles['X']
y = npzfiles['y']
X_nn = npzfiles['X_nn'] #Zero-padded dataset for Neural Net Dimensionality

output_folder = Path(RESULTS_PATH,'DifferentialCancer/')
output_folder.mkdir(exist_ok=True) #Create output folder if it does not exist 

In [82]:
pos = np.sum(y)
total = len(y)
neg  = total-pos

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

kwargs['fit__class_weight'] = class_weight

models['NeuralNetwork']=KerasClassifier(**kwargs)


In [83]:
models = {}


models['LGBM'] = LGBMClassifier(n_jobs=-1)

for name,clf in models.items():
    if name == 'QDA':
        lr = LogisticRegression(penalty='l1',solver='saga',class_weight='balanced')
        selector = SelectFromModel(lr)
    else:
        selector = 'passthrough'
    
    pipe = Pipeline(steps=[
    ('imputation',imputer),
    ('selector',selector),
    ('classifier', clf)])  
    print(name)
    
    if(name == 'NeuralNetwork'):
        report = cross_validate(pipe,X_nn,y, cv=cv, scoring=scoring)
    else:
        report = cross_validate(pipe,X,y, cv=cv, scoring=scoring)
    save_report(report,output_folder,name)
    
del models['LGBM']

LGBM
[LightGBM] [Info] Number of positive: 368, number of negative: 45
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.134708 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1500007
[LightGBM] [Info] Number of data points in the train set: 413, number of used features: 10899
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.891041 -> initscore=2.101420
[LightGBM] [Info] Start training from score 2.101420
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

# Differential Methylation Subtype (FvPTC vs CvPTC)

In [84]:
DifferentialSubtypePath = Path(THYROID_PATH,'DifferentialSubtypeData.npy')
npzfiles = np.load(DifferentialSubtypePath,allow_pickle=True)

X = npzfiles['X']
y = npzfiles['y']
X_nn = npzfiles['X_nn'] #Zero-padded dataset for Neural Net Dimensionality

output_folder = Path(RESULTS_PATH,'DifferentialSubtype/')
output_folder.mkdir(exist_ok=True) #Create output folder if it does not exist 

In [85]:
pos = np.sum(y)
total = len(y)
neg  = total-pos

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

kwargs['fit__class_weight'] = class_weight

models['NeuralNetwork']=KerasClassifier(**kwargs, reset_last_layer=True)


In [88]:
import numpy as np

for name, clf in models.items():
    print(f"--- Processing: {name} ---")
    
    if name == 'NeuralNetwork':
        # 1. Fill NaNs with 0 manually to keep exactly 485577 features
        X_nn_clean = np.nan_to_num(X_nn, nan=0.0)
        
        # 2. Run cross_validate directly on the classifier (bypassing the pipeline)
        try:
            report = cross_validate(clf, X_nn_clean, y, cv=cv, scoring=scoring, n_jobs=1)
            save_report(report, output_folder, name)
            print(f"SUCCESS: {name} finished.")
        except Exception as e:
            print(f"FAILED {name}: {e}")
            
    else:
        # Standard pipeline for other models
        selector = 'passthrough' # Add your QDA logic here if needed
        pipe = Pipeline(steps=[
            ('imputation', imputer),
            ('selector', selector),
            ('classifier', clf)
        ])
        
        try:
            report = cross_validate(pipe, X, y, cv=cv, scoring=scoring)
            save_report(report, output_folder, name)
            print(f"SUCCESS: {name} finished.")
        except Exception as e:
            print(f"FAILED {name}: {e}")

--- Processing: NeuralNetwork ---
Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 177s 2s/step - accuracy: 0.7799 - loss: 0.5625
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.7799 - loss: 0.5456
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.7799 - loss: 0.5369
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.7799 - loss: 0.5323
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.7799 - loss: 0.5301
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 273ms/step
Epoch 1/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.7778 - loss: 0.5303
Epoch 2/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.7778 - loss: 0.5299
Epoch 3/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.7778 - loss: 0.5299
Epoch 4/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.7778 - loss: 0.5304
Epoch 5/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.7778 - loss: 0.5301
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 284ms/step
Epoch 1/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 179s 2s